In [1]:
import os

In [2]:
import pandas as pd

In [3]:
persony = pd.read_json(os.path.join("data", "persony_raw.json"))

In [4]:
persony

,Jméno,Medailon,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
0,Ondřej Mašek (1000),,None,None,None,None,None,None
1,kino Světozor (100019),None,None,None,None,None,None,None
2,"AD WOOD, s.r.o. (100038)",None,None,None,None,None,None,None
3,Hypermarket Film s.r.o. (100039),None,Hypermarket Film,None,None,None,None,None
4,Hypermarket Film s.r.o. (100039),None,Hypermarket Film,None,None,None,None,None
...,...,...,...,...,...,...,...,...
35810,Josef Dvořák (9998),"herec, divadelní podnikatel",Josef Voloďa Dvořák (rodné jméno),None,25.04.1942,"Horní Cerekev, Československo",None,None
35811,Antonín Dimitrov (99996),"výtvarník, grafik, scénograf, designér",None,None,27.02.1928,"Mšecké Žehrovice, Československo",27.12.2014,"Bobcaygeon, Kanada"
35812,Čechomor (person),None,None,None,None,None,None,None
35813,Kevin Mc Kidd (person),,None,None,None,None,None,None


In [5]:
persony.loc[persony["Jméno"] == "Eva Pecková (34563)", "Datum narození"] = None
persony.loc[persony["Jméno"] == "Dal Stivens (110800)", "Datum narození"] = None
persony.loc[persony["Jméno"] == "Jaroslav Frič (129945)", "Datum narození"] = None

In [6]:
persony["Datum narození"].sample(200).unique()

array([None, '08.06.1905', '20.12.1894', '1962', '18.10.1935',
       '24. 1. 1949', '27.05.1905', '06.04.1980', '01.09.1901',
       '09.11.1923', '12.02.1931', '11.09.1889', '16.11.1960',
       '19.12.1925', '30.07.1931', '14.05.1984', '09.07.1969',
       '03.01.1909', '10.01.1909', '05.08.1941', '14.02.1963',
       '14.07.1910', '1997', '29.04.1894', '07.01.1942', '18.03.1965',
       '14.06.1957', '1990?', '20.03.1894', '1964', '22.08.1878',
       '30.11.1954', '1968', '20.04.1921', '16.11.1896', '26.01.1910',
       '26.02.1949', '1966', '1911', '11.04.1953', '21.10.1925',
       '24.07.1920', '13.11.1812', '28.09.1986', '30.10.1947', '1991',
       '23.03.1963', '04.10.1887', '189?', '19.08.1948', '22.03.1981',
       '03.02.1939', '26.05.1944', '08.09.1920', '1942', '08.12.1914',
       '16.02.1938', '26.05.1981', '30.10.1883', '14.09.1952',
       '11.06.1911', '1956', '07.09.1948', '27.02.1908', '03.01.1971',
       '29.12.1952', '12.02.1908', '10.09.1894', '13.12.1923'],


In [7]:
def odnebovat(x):
    if x:
        if "nebo" in x:
            x = x.split("nebo")[1].strip()
    return x

In [8]:
def rozdel_misto(x):
    if type(x) == str:
        try:
            mesto = x.split(",")[0]
            stat = x.split(",")[1]
        except:
            mesto = None
            stat = x
    else:
        mesto = None
        stat = None
    return [mesto, stat]

In [9]:
persony["Město narození"] = persony["Místo narození"].apply(lambda x: rozdel_misto(x)[0])
persony["Stát narození"] = persony["Místo narození"].apply(lambda x: rozdel_misto(x)[1])
persony["Město úmrtí"] = persony["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])
persony["Stát úmrtí"] = persony["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])
persony["Rok narození"] = pd.to_numeric(persony["Datum narození"].apply(lambda x: x[-4:] if isinstance(x, str) else None), errors="coerce")
persony["Rok úmrtí"] = pd.to_numeric(persony["Datum úmrtí"].apply(lambda x: x[-4:] if isinstance(x, str) else None), errors="coerce")

In [11]:
def na_data(sloupec):
    global persony
    persony["Pracovni"] = persony[sloupec]
    persony["Pracovni"] = persony["Pracovni"].apply(lambda x: odnebovat(x))
    for index, row in persony.iterrows():
        try:
            date = pd.to_datetime(row["Pracovni"], format = "%d.%m.%Y", errors="ignore")
        except ValueError:
            date = pd.to_datetime(row["Pracovni"], format = "%Y", errors="ignore")
        except ValueError:
            date = None
        persony.at[index, sloupec] = date
    persony = persony.drop(["Pracovni"], axis = 1)

In [12]:
na_data("Datum narození")
na_data("Datum úmrtí")

In [13]:
persony

,Jméno,Medailon,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Město narození,Stát narození,Město úmrtí,Stát úmrtí,Rok narození,Rok úmrtí
0,Ondřej Mašek (1000),,None,None,None,None,None,None,None,None,None,None,NaN,NaN
1,kino Světozor (100019),None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
2,"AD WOOD, s.r.o. (100038)",None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
3,Hypermarket Film s.r.o. (100039),None,Hypermarket Film,None,None,None,None,None,None,None,None,None,NaN,NaN
4,Hypermarket Film s.r.o. (100039),None,Hypermarket Film,None,None,None,None,None,None,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35810,Josef Dvořák (9998),"herec, divadelní podnikatel",Josef Voloďa Dvořák (rodné jméno),None,1942-04-25 00:00:00,"Horní Cerekev, Československo",None,None,Horní Cerekev,Československo,None,None,1942.0,NaN
35811,Antonín Dimitrov (99996),"výtvarník, grafik, scénograf, designér",None,None,1928-02-27 00:00:00,"Mšecké Žehrovice, Československo",2014-12-27 00:00:00,"Bobcaygeon, Kanada",Mšecké Žehrovice,Československo,Kanada,Kanada,1928.0,2014.0
35812,Čechomor (person),None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
35813,Kevin Mc Kidd (person),,None,None,None,None,None,None,None,None,None,None,NaN,NaN


In [14]:
persony.to_json(os.path.join("data", "persony.json"), orient="records")